In [14]:
def current_milli_time():
    return int(round(time.time() * 1000))

def prf(G, s):
    if G == E(0,1,0): 
        return 1
    return (int(G.xy()[0]) + int(G.xy()[1])) % s + 1

# Reference: EC Handbook (Sec 19.4)
def bsgs(start_pow, zeta, zeta_s, m, m_hat, left_G, right_G):
    inverse_zeta = inverse_mod(zeta, r)
    inverse_zeta_s = inverse_mod(zeta_s, r)

    List1 = dict()
    for u in range(m):
        List1[(inverse_zeta^start_pow * inverse_zeta_s^u)*left_G] = u
        
    for v in range(m_hat+1):
        if (List1.get((zeta_s^(m*v))*right_G) != None):
            print("Collision")
            u = List1[zeta_s^(m*v)*right_G]
            return u + m*v

    assert(1 == 2) # Collision must occur
        
        
def pollard_kangaroo(left_G, right_G, zeta, s, start_G, c1, c2):                                                             
    hash_table = dict()

    tame_dist, tame_walk = 0, zeta^c1 * left_G
    for j in range(s): # construct hash table for a random set of points
        hash_table[tame_walk] = j
        tame_dist += prf(F(P, s, zeta, j), s)
        tame_walk = zeta^tame_dist * P
    
    wild_dist, wild_walk = 0, zeta^c2 * right_G
    for i in range(s):
        wild_dist += prf(F(Q, s, zeta, i), s)
        wild_walk = zeta^wild_dist * right_G
        if wild_walk == tame_walk:
            print("Kangaroo Collision")
            return (tame_dist - wild_dist) % s
    
    return 0

In [15]:
import random, time

def current_milli_time():
    return int(round(time.time() * 1000))

# Cheon Algorithms to solve DLPwAI
# ---


# Toy example
p = 19
K = GF(p)
a = 1
b = -1
E = EllipticCurve(K, [a,b])
r = E(7,11).order()
d = (r-1).divisors()[1] # How is this divisor chosen such that runtime is optimal?

# Private key
alpha = 11 

# Public keys
G = E(7,11)
G_1 = alpha * G
G_d = alpha^d * G

assert(G_d == (alpha^d % r)*G)
assert(r * G_d == E(0,1,0))

for i in range(3,r):
    if (mod(i, r).is_primitive_root()):
        zeta = i
        break

        
# Cheon BSGS

start_timestamp = current_milli_time()


m = ceil(sqrt((r-1)/d))
k_0 = bsgs(0, zeta, zeta^d, m, floor((r-1)/(m*d)), G_d, G)
print(f"k_0 = {k_0}")

assert(mod(alpha^d, r) == mod(zeta^(d*k_0), r))

m = ceil(sqrt(d))
k_1 = bsgs(k_0, zeta, zeta^(int((r-1)/d)), m, floor(d/m), G_1, G)
print(f"k_1 = {k_1}")

k = k_0 + k_1*((r-1)/d)
print(f'k = {k}')

assert(alpha == mod(zeta^k, r))

alpha = mod(zeta^k, r)
print(f'The secret key found using BSGS is alpha = {alpha}')


end_timestamp = current_milli_time()
print('Runtime (in ms):', end_timestamp - start_timestamp)


# Cheon Kangaroo

# start_timestamp = current_milli_time()

# k_1 = pollard_kangaroo(G_d, G, zeta^d, ceil(sqrt(d_prime)), 2, 3) # appropriate search interval [a,b]?
# k_2 = pollard_kangaroo(G_1, zeta^k_1 * G, zeta^d_prime, ceil(sqrt(d)), 2, 3)
# alpha = zeta ^ (k_1 + k_2*d_prime)
# print("Found secret key %d using Kangaroo", alpha)

# end_timestamp = current_milli_time()
# print('Runtime (in ms):', end_timestamp - start_timestamp)

Collision
k_0 = 3
Collision
k_1 = 1
k = 12
The secret key found using BSGS is alpha = 11
Runtime (in ms): 5
